In [1]:
!pip install -q langchain-openai langchain

from google.colab import userdata
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

# Get your OpenAI API key
OPENAI_API_KEY = userdata.get("OpenAI-key")

# Initialize OpenAI and Memory
llm = OpenAI(api_key=OPENAI_API_KEY)
memory = ConversationSummaryMemory(llm=llm)

# Prompt Template
template = """
{history}
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# Create the chain using RunnableSequence
def memory_load(input_data):
    return memory.load_memory_variables(input_data)["history"]

def add_message(input_output):
    memory.save_context({"input": input_output["input"]}, {"output": input_output["output"]})
    return input_output["output"]

chain = (
    {"input": RunnablePassthrough(), "history": RunnableLambda(memory_load)}
    | prompt
    | llm
    | RunnableLambda(lambda x: {"input": input_data["input"], "output":x})
    | RunnableLambda(add_message)
)

# Example Conversation
input_data = {"input": "We examined the implications of AI on societal structures."}
result1 = chain.invoke(input_data)
print(result1)

input_data = {"input": "What were the primary topics discussed?"}
result2 = chain.invoke(input_data)
print(result2)

input_data = {"input": "Kindly, provide a summary of our discussion."}
result3 = chain.invoke(input_data)
print(result3)

print(memory.load_memory_variables({}))

<ipython-input-1-4878156c269b>:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


 Are you interested in the implications of AI on societal structures?
 The primary topics discussed were the potential impact of AI on societal structures and whether the human is interested in this topic.
 Our discussion focused on the implications of AI on societal structures. We talked about the potential impact of AI on various aspects of society, such as the economy and workforce. We also discussed the human's interest in the topic and the need for continued research and regulation to ensure responsible development and use of AI.
{'history': "\nThe human and AI discuss the implications of AI on societal structures, including the potential impact and the human's interest in the topic. They also touch on the need for continued research and regulation to ensure responsible development and use of AI. The human asks for a summary of their discussion, to which the AI provides a detailed overview of their conversation."}
